In [25]:
import pandas as pd
import numpy as np

In [ ]:
# lat
# lng


In [ ]:
DICT_STANDARDIZE_NAMING = {
    "latitude": "lat",
    "longitude": "lng",
    "website": "url",
}

# Pre-processing

In [38]:
def get_steel_plants():
    """
        source: https://globalenergymonitor.org/projects/global-steel-plant-tracker/
    """
    df_steel = (
        pd
        .read_csv("../datasets/steel-plant-infrastructure-dataset.csv")
        .assign(asset_type="steel-plant")
    )
    
    return df_steel

In [40]:
get_steel_plants().head(2)

,project,project_other,parent,owner,ownership,province,country,region,status,steel_capacity,iron_capacity,process,equipment,url,lat,lng,asset_type
0,GHC Emirates Steel Industries plant,حديد الإمارات (Arabic),General Holding Company (GHC),Emirates Steel Industries Co. PJS,state-owned,Abu Dhabi,United Arab Emirates,Western Asia,operating,3100,1900,integrated_DRI,"DRI, EAF",https://www.gem.wiki/GHC_Emirates_Steel_Indust...,24.322033,54.467987,steel-plant
1,ArcelMittal Acindar steel plant,Planta siderúrgica Acindar (Spanish),ArcelorMittal,Acindar Industria Argentina de Aceros SA (Acin...,private,Santa Fe,Argentina,Latin America and the Caribbean,operating,1750,1000,integrated_DRI,"DRI, EAF",https://www.gem.wiki/ArcelorMittal_Acindar_ste...,-33.249936,-60.300415,steel-plant


In [34]:
def get_coal_mines():
    """
        source: https://globalenergymonitor.org/projects/global-coal-mine-tracker/
    """
    df_coal = (
        pd
        .read_csv("../datasets/coal-mine-infrastructure-dataset.csv")
        .assign(asset_type="coal_mine")
    )
    
    return df_coal

In [35]:
get_coal_mines().head(2)

,project,status,owner,operator,output,size,coal_type,coal_grade,type,workforce,country,region,url,lat,lng,asset_type
0,Shadashak Coal Mine,operating,North Coal Enterprise (NCE),North Coal Enterprise (NCE),1.1,NaN,Bituminous,met,underground,NaN,Afghanistan,Africa & Middle East,https://www.gem.wiki/Shadashak_Coal_Mine,35.863255,67.596674,coal_mine
1,Airly Coal Mine,operating,Centennial Coal,Centennial Airly Pty Ltd,1.8,34.0,Bituminous,thermal,underground,140,Australia,Australia & NZ,https://www.gem.wiki/Airly_Mine,-33.129008,149.975000,coal_mine


In [42]:
def get_fossil_pipelines():
    """
        source: https://globalenergymonitor.org/projects/global-fossil-infrastructure-tracker/tracker-map/
    
        No lat/lng for pipelines
        No route for end point of pipelines
    """
    df_fossil = (
        pd
        .read_csv("../datasets/fossil-pipelines-infrastructure-dataset.csv")
        .assign(asset_type="fossil")
    )
    
    return df_fossil

In [46]:
get_fossil_pipelines().query("lat==lat").head(2)

,project,owner,status,countries,type,lat,lng,route,url,capacity1,capacity_unit1,asset_type
2760,Fourchon LNG Terminal Phase 1,EnergyWorld,proposed,USA,LNG,29.105833,-90.194444,NaN,https://www.gem.wiki/Fourchon_LNG_Terminal,2.00,mtpa,fossil
2761,Fourchon LNG Terminal Phase 2,EnergyWorld,proposed,USA,LNG,29.105833,-90.194444,NaN,https://www.gem.wiki/Fourchon_LNG_Terminal,3.00,mtpa,fossil


In [45]:
get_fossil_pipelines().query("route==route").head(2)

,project,owner,status,countries,type,lat,lng,route,url,capacity1,capacity_unit1,asset_type
0,Ionian Adriatic Gas Pipeline (IAP),"SOCAR, 100.00%",proposed,"Albania, Bosnia and Herzegovina, Croatia",Gas,NaN,NaN,"43.5995, 16.3946: 43.6098, 16.5395: 43.596, 16...",https://www.gem.wiki/Ionian_Adriatic_Gas_Pipeline,"82,828",BOE/d,fossil
1,Eagle FLNG Pipeline,Gruppo Falcione,cancelled,"Italy, Albania",Gas,NaN,NaN,"40.683333,19.483333:40.645137,17.609292",https://www.gem.wiki/Eagle_FLNG_Pipeline,"165,655.00",BOE/d,fossil


In [36]:
def get_power_plant():
    """
    source: https://wiki.openstreetmap.org/wiki/Tag:power%3Dplant
    
    Plant sources available:
        ['hydro', 'waste', 'gas', nan, 'wind', 'oil', 'coal', 'biofuel',
       'solar;diesel', 'solar', 'oil;gas', 'biomass;oil', 'biomass',
       'gas;oil', 'biogas', 'nuclear', 'battery',
       'abandoned_mine_methane;oil'
    
    """
    DIRTY_POWER_PLANTS = ['gas', 'oil', 'coal', 'biofuel', 'oil;gas', 'biomass;oil', 'biomass',
       'gas;oil', 'biogas', 'abandoned_mine_methane;oil']
    
    df_power_plant = (
        pd
        .read_csv("../datasets/power-plant-infrastructure-dataset.csv")
        .dropna(axis=1, how="all")
        .rename(columns={'plant:source': 'plant_source'})
        .assign(asset_type="power_plant")
        .query("plant_source in @DIRTY_POWER_PLANTS")
        # matches coal mine dataset
        .assign(type= lambda s: np.where(s.underground == 'yes', 'underground', "surface"))
    )
    
    return df_power_plant

In [37]:
get_power_plant().head(2)

,latitude,longitude,geometry,@id,generator:source,power,generator:method,addr:state,ele,gnis:county_name,...,design,height,material,structure,tower:type,generator:output:hot_water,ref:bna,plant:output:heat,asset_type,type
6,42.506389,-89.032222,Point,node/367811131,gas,plant,NaN,WI,230,Rock,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,power_plant,surface
79,37.428053,24.938588,Point,node/2685179475,NaN,plant,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,power_plant,surface
